# NLP筆記

## Clean data
在做自然語言處理時第一步驟就是要清洗資料
> The number one rule we follow is: “Your model will only ever be as good as your data.”

### 以英文來說:
1. 先將文本全部轉成小寫
2. 再將句子做 Tokenize，也就是將每個英文單字做斷詞
3. 設計stopword[1]，移除對於這次model訓練不相關的標點符號或是單字(如果有必要)
4. 移除拼寫錯誤或是非正規語法單字(e.g. “cool”/”kewl”/”cooool”)
5. lemmatization(詞形還原)(e.g.1  “am”, “are”, and “is” to a common form such as “be”)(e.g.2 動詞時態)



### 以中文來說:
1. 先做斷詞，可以使用jieba或是snownlp等支援中文的分詞系統
2. 設計stopword，移除對於這次model訓練不相關的標點符號或是詞語(e.g. 介係詞、語助詞)
3. 移除拼寫錯誤或是非正規語法單字(火星文、注音文，不過如果是拿正規的語料基本上不用煩惱這個問題，或是針對特殊用語創建語料庫，方便分詞系統做斷詞)

<br>
<br>


註[1]stopword:在信息檢索中，為節省存儲空間和提高搜索效率，在處理自然語言數據（或文本）之前或之後會自動過濾掉某些字或詞，這些字或詞即被稱為Stop Words。
通常意義上，停用詞大致分為兩類。一類是人類語言中包含的功能詞，這些功能詞極其普遍，與其他詞相比，功能詞沒有什麼實際含義，比如'the'、'is'、'at'、'which'、'on'等。但是對於搜尋引擎來說，當所要搜索的短語包含功能詞，特別是像The Who、The The或Take That等複合名詞時，停用詞的使用就會導致問題。另一類詞包括詞彙詞，比如'want'等，這些詞應用十分廣泛，但是對這樣的詞搜尋引擎無法保證能夠給出真正相關的搜索結果，難以幫助縮小搜索範圍，同時還會降低搜索的效率，所以通常會把這些詞從問題中移去，從而提高搜索性能。

舉例來說，當今天是要做文本分類，要將某段文字辨認是哪位作者寫的，stopword就推薦使用，甚至可以做tf-idf，找出某位作者最常使用且可以跟其他作者區隔的詞語。

如果今天是要做敘述生成，那麼stopword就不應該使用，使用stopword會使那些字沒辦法生成。

---

## Find a good data representation

不管是機器學習還是深度學習，Input都必須是數值。

以圖像來說，Input是一個矩陣，而在每個color channel中都有pixel去表示那個點的顏色

而在做自然語言處理，我們必須為這些字建一個數值表示法
### One-hot encoding (Bag of Words)
我們可以依照dataset建立一個所有獨特詞彙的詞彙表，並將唯一index與詞彙表中的每個詞彙相關聯。
然後，每個句子都被表示為一個跟單字數量一樣長的列表。
在此列表中的每個索引處，我們標記給定單詞在我們的句子中出現的次數。這叫做[Bag of Words](https://en.wikipedia.org/wiki/Bag-of-words_model)模型，因為它是一種完全忽略我們句子中單詞順序的表示。

![](https://imgur.com/fI7uwik.png)

假如今天的dataset詞彙量有2萬個單字，這意味著每個句子都會被表示為長度為20,000的向量。該向量將包含大部分0，因為每個句子只包含我們詞彙的一小部分。

直觀來說這在訓練上效果不會到非常理想:
1. 某些字可能出現過於頻繁，而變成noise
2. 矩陣太過於稀疏，向量的維度會隨著句子的詞的數量類型增大而增大
3. 因為忽略句子中單詞順序的表示，也就是說任意兩個詞之間都是孤立的，根本無法表示出在語義層面上詞語詞之間的相關信息，所以對於語句理解上會有落差。
---

### TF-IDF

[教學1](https://taweihuang.hpd.io/2017/03/01/tfidf/)

[教學2](https://blog.csdn.net/u012052268/article/details/79560768)

TF-IDF（Term Frequency-InversDocument Frequency）是一種常用於信息處理和數據挖掘的加權技術。該技術採用一種統計方法，根據字詞的在文本中出現的次數和在整個語料中出現的文檔頻率來計算一個字詞在整個語料中的重要程度。它的優點是能過濾掉一些常見的卻無關緊要本的詞語，同時保留影響整個文本的重要字詞。 
* TF（Term Frequency）表示某個關鍵詞在整篇文章中出現的頻率。 
* IDF（InversDocument Frequency）表示計算倒文本頻率。文本頻率是指某個關鍵詞在整個語料所有文章中出現的次數。倒文檔頻率又稱為逆文檔頻率，它是文檔頻率的倒數，主要用於降低所有文檔中一些常見卻對文檔影響不大的詞語的作用。 

計算方法：通過將局部分量（詞頻）與全局分量（逆文檔頻率）相乘來計算tf-idf，並將所得文檔標準化為單位長度。文件中的文檔中的非標準權重的公式，如圖：

![](https://imgur.com/k0d1nVW.png)

分開的步驟 
1. 計算詞頻 　　

     詞頻 = 某個詞在文章中出現的總次數/文章的總詞數
     
     ![](https://imgur.com/rwkdN4A.png)

2. 計算逆文檔頻率 

   逆文檔頻率（IDF） = log（詞料庫的文檔總數/包含該詞的文檔數+1）
   
知道了”詞頻”（TF）和”逆文檔頻率”（IDF）以後，將這兩個值相乘，就得到了一個詞的TF-IDF值。某個詞對文章的重要性越高，它的TF-IDF值就越大。所以，排在最前面的幾個詞，就是這篇文章的關鍵詞。
   

#### TF-IDF應用
TF-IDF 常被用在3個地方，一個是作為 baseline model的特徵 (feature)，比如說作文件分類 (text classification) 時，就會把 tf 跟 idf 都當作文件的特徵(所以一篇文章總夠會有 2T個特徵)，去跑分類模型，作為 baseline。有時候會把一個詞彙對於每篇的文章的 tf-idf 值當作該詞彙的特徵，去跑文字的分群。還有一個就是如果我們想要用word2vec得出來的詞向量去表示一個文件的話，可以用tf-idf的值當作權重，再把該文件中每個詞向量用tf-idf當作權重加起來，效果也很不錯

---

### Word Embedding(詞嵌入)

將向量與單詞關聯的另一種流行且強大的方式是使用密集的“詞向量(word vectors)”，也稱為“詞嵌入(word embeddings)”。

雖然通過one-hot編碼獲得的向量是二元，稀疏（大部分由零構成）而且維度會變成非常的高（與詞彙表中單詞的數量相同, 也就是如果有1000個不重覆的單詞, 那每個單詞就會變成1000維的向量），然而“詞嵌入(word embeddings)”卻可以用低維度的浮點向量 （即“密集”向量，而不是稀疏向量）來表示相同的一個單詞。

![](https://imgur.com/TblnOdk.png)

與透過one-hot編碼獲得的單詞向量不同地方在於“單詞嵌入(word embeddings)”是從數據中學習得來的。通常在處理非常大的詞彙表時，我們通常會看到256維，512維或1024維的“詞嵌入(word embeddings)”層。

另一方面，one-hot編碼單詞通常導致20,000維或更高維的向量（假設我們需要處理 20,000個詞彙的詞彙表的任務）。因此從某種角度來說，“詞嵌入(word embeddings)”可以用將更多信息包裝到更少的維度中。

![](https://imgur.com/J4bWnf2.png)


如果要做一個NLP任務且需要用到詞向量時，有兩個方法
1. 自己訓練自己的詞向量(train壞的機率高train的時間久，但如果train成功，對於自己所使用的dataset是比使用別人train好的詞向量更適合)
   舉例來說，把“詞嵌入(word embeddings)”與我們關心的主要任務模型（例如文檔分類或情感預測）一起設計在模型的結構中一起訓練共同學習。 在這樣的設計裡，我們將從隨機詞向量開始，然後學習我們的詞向量(word vector)，就像學習神經網絡的權重一樣。

2. 選擇使用別人訓練好的word embeddings，注意，得使用相同語料內容領域的詞向量，也稱作Pre-trained word embedding


#### 利用嵌入層(embedding layer)來學習單詞嵌入(word embeddings)
將單詞與密集向量相關聯的最簡單方法是選擇一種隨機向量(vector at random)。這種方法的問題是，這樣的嵌入空間(embedding space)將沒有什麼結構的關聯性：例如，"accurate"和"exact"這兩個字可能會是完全不同的嵌入(embeddings)，深度神經網絡很難理解這樣一個嘈雜的，非結構化的嵌入空間。

為了得到更抽象一點：不同詞向量(word vectors)之間的幾何關係應該反映這些詞之間的語義關係。詞嵌入(word embeddings)意味著將人類語言映射到幾何空間(geometric space)。例如，在一個合理的嵌入空間中，我們期望 同義詞被嵌入到相似的單詞向量中，並且通常我們會期望任意兩個之間的幾何距離(例如L2距離)單詞向量與相關單詞的語義距離相關(單詞意味著非常不同的事物將嵌入到點中 彼此相距很遠，而相關的詞會更接近)。即使超出了距離，我們可能會希望在嵌入空間中方向 是有意義的。

在現實世界的詞嵌入空間中，有意義的幾何變換的常見例子是“性別向量(gender vectors)”和“複數向量(plural vector)”。例如，通過給“國王(king)”向量添加“女性(female)”向量，會獲得“女王(queen)”向量。通過添加“複數(plural)矢量”，可以獲得“國王們(kings)”。 字嵌入空間通常具有數千個這樣的可解釋和潛在有用的向量。

![](https://imgur.com/e6hn1F4.png)

是否有一些“完美”的單詞嵌入空間可以完美地映射人類語言，並可用於任何自然語言處理 任務？ 或許吧，但無論如何，我們還沒有訓練出這樣類型的東西出來。另外，不同的語言有著不同的結構，因為語言是特定文化和特定背景的反映。

那麼從實務的角度來說，到底什麼樣的詞嵌入空間才算的上是一個好的詞嵌入？ 它很大程度上取決於我們想要進行的任務：一個“電影評論情感分析模型”的的詞嵌入空間可能與“法律文件分類模型”的詞嵌入空間看起來有很大不同，因為某些語義關係的重要性會因任務或要解決的問題而有所差異。

嵌入層(embedding layer)最好的理解是: 將整數索引（代表特定單詞）映射到密集向量的字典物件中。輸入整數，它查找一個內部字典，並返回相關的向量。

當我們實例化一個Embedding層的時候，它的權重（它的內部符標向量字典）最初是隨機的，就像任何其他神經層。在訓練過程中，這些單詞向量將通過反向傳播逐漸調整，將嵌入空間構建成一些下游模式可以利用的東西。一旦經過充分訓練，我們的嵌入空間將顯示出許多結構(一種專門針對我們正在訓練模型的特定問題結構)。

#### 使用預先訓練的詞嵌入(pre-trained word embeddings)
有時候，我們只有很少的訓練數據可供使用，因此無法單獨使用我們的數據來學習適合特定任務的詞彙嵌入。那該怎麼辦？

我們可以加載預先訓練好的詞嵌入(pre-trained word embeddings)。在自然語言處理中使用預先訓練的單詞嵌入背後的原理與在圖像分類中使用預先訓練的模型非常相似：我們沒有足夠的數據來自己學習真正強大的模型，但我們需要相當通用的特徵，即常見的視覺特徵或語義特徵。在這種情況下，重覆使用在不同問題上學習到的特徵是有意義的。

一般都會使用詞語出現(word occurrency)統計量來計算這種詞嵌入，但也有人用神經網絡來訓練詞嵌入。 最著名和成功的詞嵌入方案之一：2013年由Mikolov在谷歌開發的Word2Vec算法。


## Leveraging semantics

### Word2Vec

![](https://imgur.com/kB5f0mV.png)

一般進行NLP(自然語言處理)時，資料最小的細粒度是語詞(word)，語詞組成句子，句子再組成句子，篇章，文檔。所以很多處理NLP的問題的前處理程序，首先就要思考語詞(word)的處理。

舉個簡單的例子，判斷一個詞的詞性，是動詞還是名詞。用機器學習的思路，我們有一系列樣本（x，y），這裡x是語詞，y是它們的詞性，我們要構建f(x) -> y的映射，但這裡的數學模型f（比如神經網絡或SVM）只接受數值型輸入，而NLP裡的語詞，是人類的語言抽象表示，是以符號形式來表現的（比如中文，英文，拉丁文等等），所以需要把他們轉換成數值形式，或者說"嵌入"到一個數學空間裡，這種嵌入方式，就叫"詞嵌入(word embedding)"，而**Word2vec**，就是詞嵌入(word embedding)的一種作法。

在NLP中，把x看做一個句子裡的一個語詞，y是這個語詞的上下文(context)語詞，那麼這裡的f，便是NLP中經常出現的「語言模型」（language model），這個模型的目的，就是判斷（x，y）這個樣本，是否符合自然語言的法則，更通俗點說是：語詞x和語詞y放在一起時，像不像是人寫的話或寫的句子。

Word2vec正是來自於這個想法，但它的最終目的，不是要把f訓練得多麼完美，而是關心模型訓練完後的副產物-"模型參數"（這里指的是神經網絡的權重），並將這些參數，作為輸入x的某種向量化的表示，這個向量便叫做"詞向量(word vector)"。

Word2Vec是一種為單詞查找連續嵌入的技術。它通過閱讀大量文本來學習，並記憶哪些詞傾向於出現在相似的情境中。在對足夠的數據進行訓練之後，它會為詞彙表中的每個單詞生成一個300維矢量，並且相似含義的單詞彼此更接近。

為我們的分類器獲得句子嵌入的一種快速方法是對我們句子中所有單詞的Word2Vec分數取平均值。這是一個像以前一樣的詞袋方法，但是這次我們只丟失了句子的語法，同時保留了一些語義信息。

![](https://imgur.com/ZdD3YjB.png)